In [1]:
import duckdb

In [10]:
duckdb.sql("""
create or replace table Dirty as select row_number() over () as id, * from read_csv('PV_Module_List_Full_Data_ADA.2026-02-11.csv');
describe Dirty;
""")

┌─────────────────────────────────────────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│                               column_name                               │ column_type │  null   │   key   │ default │  extra  │
│                                 varchar                                 │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                                                                      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Manufacturer                                                            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Model Number                                                            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description                                                             │ VARCHAR     │ 

In [4]:
duckdb.sql("""
create or replace table EntityCode as select * from read_csv('../EntityCodes.csv');
describe EntityCode;
""")

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ProdMfr_Value │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Email_Value   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ EntityCode    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└───────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

### ProdMfr

In [6]:
ob_prod_mfr = duckdb.sql("""
select id, "Manufacturer" as ProdMfr_Value from Dirty;
""")
ob_prod_mfr.show(max_width=120)

┌───────┬──────────────────────────┐
│  id   │      ProdMfr_Value       │
│ int64 │         varchar          │
├───────┼──────────────────────────┤
│     1 │ Ablytek                  │
│     2 │ Ablytek                  │
│     3 │ Ablytek                  │
│     4 │ Ablytek                  │
│     5 │ Ablytek                  │
│     6 │ Advance Power            │
│     7 │ Advance Power            │
│     8 │ Advance Power            │
│     9 │ Advance Power            │
│    10 │ Advance Power            │
│     · │       ·                  │
│     · │       ·                  │
│     · │       ·                  │
│  9991 │ Mission Solar Energy LLC │
│  9992 │ Mission Solar Energy LLC │
│  9993 │ Mission Solar Energy LLC │
│  9994 │ Mission Solar Energy LLC │
│  9995 │ Mission Solar Energy LLC │
│  9996 │ Mission Solar Energy LLC │
│  9997 │ Mission Solar Energy LLC │
│  9998 │ Mission Solar Energy LLC │
│  9999 │ Mission Solar Energy LLC │
│ 10000 │ Mission Solar Energy LLC │
├

### ProdCode
If build the ProdCode, we first need to match the CEC manufacturer names to those given by SunSpec.

### Description

In [7]:
ob_description = duckdb.sql("""
select id, Description as Description_Value from Dirty;
""")
ob_description.show(max_width=120)

┌───────┬────────────────────────────────────────────────────────────────────────────────────────┐
│  id   │                                   Description_Value                                    │
│ int64 │                                        varchar                                         │
├───────┼────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │ 270 W Monocrystalline Module                                                           │
│     2 │ 275 W Monocrystalline Module                                                           │
│     3 │ 280 W Monocrystalline Module                                                           │
│     4 │ 285 W Monocrystalline Module                                                           │
│     5 │ 290 W Monocrystalline Module                                                           │
│     6 │ 210 W Polycrystalline Module                                                           │
│     7 │ 

### ProdCertifications
This is a list.

In [103]:
ob_prod_certifications = []

### ProdCertification for "Safety Certification"
Here there is extra white space, and improper values.

In [102]:
duckdb.sql("""
select distinct "Safety Certification" from Dirty
""")

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                                    Safety Certification                                     │
│                                           varchar                                           │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│ UL 1741                                                                                     │
│ UL 61730                                                                                    │
│ NULL                                                                                        │
│ UL 61730, UL 1703                                                                           │
│ UL 61731                                                                                    │
│ Ul 61730                                                                                    │
│ 501 W, 120 half-cut cell double glass 

The improper value starts with '501 W'.
There is only one module with this improper value.
The [installation manual of Thornova Solar](https://www.thornova-solar.com/attached/file/na/manuals/Installation_Manual.pdf) says that all their modules have UL 61730 certifications, so we will use this value.

In [100]:
duckdb.sql("""
select * from Dirty where "Safety Certification" = '501 W, 120 half-cut cell double glass bifacial monocrystalline module, 1500V max system Vdc';
""").show(max_width=1)

┌───────┬────────────────┬───────────────┬───┬──────────────────┬─────────────┐
│  id   │  Manufacturer  │ Model Number  │ … │ CEC Listing Date │ Last Update │
│ int64 │    varchar     │    varchar    │   │       date       │    date     │
├───────┼────────────────┼───────────────┼───┼──────────────────┼─────────────┤
│ 16596 │ Thornova Solar │ TS-BGT60(500) │ … │ 2025-06-02       │ NULL        │
├───────┴────────────────┴───────────────┴───┴──────────────────┴─────────────┤
│ 1 rows                                                 38 columns (5 shown) │
└─────────────────────────────────────────────────────────────────────────────┘



In [105]:
ob_prod_certification = duckdb.sql("""
select distinct
    id,
    case "Safety Certification"
        when 'UL 1703' then 'UL1703_2002' 
        when 'UL 1703 ' then 'UL1703_2002' 
        when 'UL 1741' then null 
        when 'Ul 61730' then 'UL61730_2017'
        when 'UL 61730' then 'UL61730_2017'
        when 'UL 61731' then 'UL61730_2017'
        when 'UL 61730, UL 1703' then unnest(['UL1703_2002', 'UL61730_2017'])
        when '501 W, 120 half-cut cell double glass bifacial monocrystalline module, 1500V max system Vdc' then 'UL61730_2017'
        else "Safety Certification"
    end as CertificationStandard_Value
from Dirty;
""")
ob_prod_certifications.append(ob_prod_certification)
ob_prod_certification

┌───────┬─────────────────────────────┐
│  id   │ CertificationStandard_Value │
│ int64 │           varchar           │
├───────┼─────────────────────────────┤
│    61 │ UL61730_2017                │
│    68 │ UL61730_2017                │
│    69 │ UL61730_2017                │
│    72 │ UL61730_2017                │
│   117 │ UL1703_2002                 │
│   132 │ UL61730_2017                │
│   148 │ UL61730_2017                │
│   171 │ UL1703_2002                 │
│   230 │ UL1703_2002                 │
│   253 │ UL1703_2002                 │
│    ·  │      ·                      │
│    ·  │      ·                      │
│    ·  │      ·                      │
│  6575 │ UL1703_2002                 │
│  6591 │ UL1703_2002                 │
│  6592 │ UL1703_2002                 │
│  6634 │ UL61730_2017                │
│  6638 │ UL61730_2017                │
│  6649 │ UL61730_2017                │
│  6652 │ UL61730_2017                │
│  6679 │ UL61730_2017                │


### ProdCertification for "Design Qualification Certification (Optional Submission) IEC 61215:2016"
Since we know the certification, we only need to extract the date.

In [106]:
duckdb.sql("""
select distinct "Design Qualification Certification (Optional Submission) IEC 61215:2016" from Dirty
""")

┌─────────────────────────────────────────────────────────────────────────┐
│ Design Qualification Certification (Optional Submission) IEC 61215:2016 │
│                                 varchar                                 │
├─────────────────────────────────────────────────────────────────────────┤
│ 03.08.2021                                                              │
│ 16.03.2020                                                              │
│ 18.02.2020                                                              │
│ 11/21/2022 [IEC 61215:2021]                                             │
│ 29.11.2022                                                              │
│ 23.03.2022                                                              │
│ 23.07.2020                                                              │
│ No Information Submitted                                                │
│ 31.10.2019                                                              │
│ 29.06.2022

All the date information is before the first white space.

In [110]:
duckdb.sql("""
select distinct
    split("Design Qualification Certification (Optional Submission) IEC 61215:2016", ' ')[2]
from Dirty;
""")

┌──────────────────────────────────────────────────────────────────────────────────────────┐
│ split("Design Qualification Certification (Optional Submission) IEC 61215:2016", ' ')[2] │
│                                         varchar                                          │
├──────────────────────────────────────────────────────────────────────────────────────────┤
│ [IEC                                                                                     │
│ Information                                                                              │
│ NULL                                                                                     │
└──────────────────────────────────────────────────────────────────────────────────────────┘

Therefore, we can take this part of the string, and try to cast it to a date.

In [135]:
duckdb.sql("""
with has_date_str as (
    select
        id,
        split("Design Qualification Certification (Optional Submission) IEC 61215:2016", ' ')[1] as date_str,
    from Dirty
)
select distinct
    'IEC61215_2016' as CertificationStandard_Value,
    case
        when try_strptime(date_str, '%d.%m.%Y') is not null then cast(strptime(date_str, '%d.%m.%Y') as string)
        when try_strptime(date_str, '%m/%d/%Y') is not null then cast(strptime(date_str, '%m/%d/%Y') as string)
        else date_str
    end as CertificationDate_Value,
from has_date_str order by CertificationDate_Value
""")

┌─────────────────────────────┬─────────────────────────┐
│ CertificationStandard_Value │ CertificationDate_Value │
│           varchar           │         varchar         │
├─────────────────────────────┼─────────────────────────┤
│ IEC61215_2016               │ 2018-02-28 00:00:00     │
│ IEC61215_2016               │ 2018-03-29 00:00:00     │
│ IEC61215_2016               │ 2018-09-04 00:00:00     │
│ IEC61215_2016               │ 2018-10-26 00:00:00     │
│ IEC61215_2016               │ 2018-11-08 00:00:00     │
│ IEC61215_2016               │ 2018-12-20 00:00:00     │
│ IEC61215_2016               │ 2019-02-18 00:00:00     │
│ IEC61215_2016               │ 2019-03-08 00:00:00     │
│ IEC61215_2016               │ 2019-04-09 00:00:00     │
│ IEC61215_2016               │ 2019-05-05 00:00:00     │
│       ·                     │          ·              │
│       ·                     │          ·              │
│       ·                     │          ·              │
│ IEC61215_201

In [161]:
ob_prod_certification = duckdb.sql("""
with has_date_str as (
    select
        id,
        split("Design Qualification Certification (Optional Submission) IEC 61215:2016", ' ')[1] as date_str,
    from Dirty
)
select
    id,
    'IEC61215_2016' as CertificationStandard_Value,
    case
        when try_strptime(date_str, '%d.%m.%Y') is not null then cast(strptime(date_str, '%d.%m.%Y') as date)
        when try_strptime(date_str, '%m/%d/%Y') is not null then cast(strptime(date_str, '%m/%d/%Y') as date)
    end as CertificationDate_Value,
from has_date_str
""")
ob_prod_certifications.append(ob_prod_certification)
ob_prod_certification

┌───────┬─────────────────────────────┬─────────────────────────┐
│  id   │ CertificationStandard_Value │ CertificationDate_Value │
│ int64 │           varchar           │          date           │
├───────┼─────────────────────────────┼─────────────────────────┤
│     1 │ IEC61215_2016               │ NULL                    │
│     2 │ IEC61215_2016               │ NULL                    │
│     3 │ IEC61215_2016               │ NULL                    │
│     4 │ IEC61215_2016               │ NULL                    │
│     5 │ IEC61215_2016               │ NULL                    │
│     6 │ IEC61215_2016               │ NULL                    │
│     7 │ IEC61215_2016               │ NULL                    │
│     8 │ IEC61215_2016               │ NULL                    │
│     9 │ IEC61215_2016               │ NULL                    │
│    10 │ IEC61215_2016               │ NULL                    │
│     · │       ·                     │  ·                      │
│     · │ 

### ProdCertification for "Performance Evaluation (Optional Submission) IEC 61853-1:2011"
Since we know the certification, we only need to extract the date.

In [138]:
duckdb.sql("""
select distinct "Performance Evaluation (Optional Submission) IEC 61853-1:2011" from Dirty
""")

┌───────────────────────────────────────────────────────────────┐
│ Performance Evaluation (Optional Submission) IEC 61853-1:2011 │
│                            varchar                            │
├───────────────────────────────────────────────────────────────┤
│ 28.05.2020                                                    │
│ 18.01.2022                                                    │
│ No Information Submitted                                      │
│ 31.05.2022                                                    │
│ NULL                                                          │
└───────────────────────────────────────────────────────────────┘

In [142]:
ob_prod_certification = duckdb.sql("""
with has_date_str as (
    select
        id,
        split("Performance Evaluation (Optional Submission) IEC 61853-1:2011", ' ')[1] as date_str,
    from Dirty
)
select
    id,
    'IEC61853_1_2011' as CertificationStandard_Value,
    case
        when try_strptime(date_str, '%d.%m.%Y') is not null then cast(strptime(date_str, '%d.%m.%Y') as date)
    end as CertificationDate_Value,
from has_date_str
""")
ob_prod_certifications.append(ob_prod_certification)
ob_prod_certification

┌───────┬─────────────────────────────┬─────────────────────────┐
│  id   │ CertificationStandard_Value │ CertificationDate_Value │
│ int64 │           varchar           │          date           │
├───────┼─────────────────────────────┼─────────────────────────┤
│     1 │ IEC61853_1_2011             │ NULL                    │
│     2 │ IEC61853_1_2011             │ NULL                    │
│     3 │ IEC61853_1_2011             │ NULL                    │
│     4 │ IEC61853_1_2011             │ NULL                    │
│     5 │ IEC61853_1_2011             │ NULL                    │
│     6 │ IEC61853_1_2011             │ NULL                    │
│     7 │ IEC61853_1_2011             │ NULL                    │
│     8 │ IEC61853_1_2011             │ NULL                    │
│     9 │ IEC61853_1_2011             │ NULL                    │
│    10 │ IEC61853_1_2011             │ NULL                    │
│     · │        ·                    │  ·                      │
│     · │ 

### ModuleElectRatings
This is a list.

In [143]:
ob_module_elect_ratings = []

### ModuleElectRating for "Nameplate (...)"

In [158]:
ob_power_stc = duckdb.sql("""
select
    id,
    'W' as PowerSTC_Unit,
    "Nameplate Pmax (W)" as PowerSTC_Value,
from Dirty;
""")
ob_power_stc

┌───────┬───────────────┬────────────────┐
│  id   │ PowerSTC_Unit │ PowerSTC_Value │
│ int64 │    varchar    │     double     │
├───────┼───────────────┼────────────────┤
│     1 │ W             │          270.0 │
│     2 │ W             │          275.0 │
│     3 │ W             │          280.0 │
│     4 │ W             │          285.0 │
│     5 │ W             │          290.0 │
│     6 │ W             │          210.0 │
│     7 │ W             │          215.0 │
│     8 │ W             │          220.0 │
│     9 │ W             │          225.0 │
│    10 │ W             │          225.0 │
│     · │ ·             │            ·   │
│     · │ ·             │            ·   │
│     · │ ·             │            ·   │
│  9991 │ W             │          360.0 │
│  9992 │ W             │          365.0 │
│  9993 │ W             │          365.0 │
│  9994 │ W             │          365.0 │
│  9995 │ W             │          365.0 │
│  9996 │ W             │          365.0 │
│  9997 │ W

In [160]:
ob_module_elect_rating = duckdb.sql("""
select
    id,
    'STC' as ModuleRatingCondition_Value,
    'W' as PowerDC_Unit,
    "Nameplate Pmax (W)" as PowerDC_Value,
    'A' as CurrentShortCircuit_Unit,
    "Nameplate Isc (A)" as CurrentShortCircuit_Value,
    'V' as VoltageOpenCircuit_Unit,
    "Nameplate Voc (V)" as VoltageOpenCircuit_Value,
    'A' as CurrentAtMaximumPower_Unit,
    "Nameplate Ipmax (A)" as CurrentAtMaximumPower_Value,
    'V' as VoltageAtMaximumPower_Unit,
    "Nameplate Vpmax (V)" as VoltageAtMaximumPower_Value,
from Dirty;
""")
ob_module_elect_ratings.append(ob_module_elect_rating)
ob_module_elect_rating.show(max_width=120)

┌───────┬──────────────────────┬──────────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│  id   │ ModuleRatingCondit…  │ PowerDC_Unit │ … │ CurrentAtMaximumPo…  │ VoltageAtMaximumPo…  │ VoltageAtMaximumPo…  │
│ int64 │       varchar        │   varchar    │   │        double        │       varchar        │        double        │
├───────┼──────────────────────┼──────────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│     1 │ STC                  │ W            │ … │                 8.81 │ V                    │                30.72 │
│     2 │ STC                  │ W            │ … │                 8.88 │ V                    │                30.99 │
│     3 │ STC                  │ W            │ … │                 8.96 │ V                    │                31.26 │
│     4 │ STC                  │ W            │ … │                 9.04 │ V                    │                31.53 │
│     5 │ STC                  │

### ModuleElectRating for "PTC"

In [151]:
ob_module_elect_rating = duckdb.sql("""
select
    id,
    'PTC' as ModuleRatingCondition_Value,
    'W' as PowerDC_Unit,
    "PTC" as PowerDC_Value,
from Dirty;
""")
ob_module_elect_ratings.append(ob_module_elect_rating)
ob_module_elect_rating

┌───────┬─────────────────────────────┬──────────────┬───────────────┐
│  id   │ ModuleRatingCondition_Value │ PowerDC_Unit │ PowerDC_Value │
│ int64 │           varchar           │   varchar    │    double     │
├───────┼─────────────────────────────┼──────────────┼───────────────┤
│     1 │ PTC                         │ W            │         242.1 │
│     2 │ PTC                         │ W            │         246.7 │
│     3 │ PTC                         │ W            │         251.3 │
│     4 │ PTC                         │ W            │         256.0 │
│     5 │ PTC                         │ W            │         260.6 │
│     6 │ PTC                         │ W            │         186.8 │
│     7 │ PTC                         │ W            │         191.3 │
│     8 │ PTC                         │ W            │         195.9 │
│     9 │ PTC                         │ W            │         200.7 │
│    10 │ PTC                         │ W            │         200.5 │
│     

### ModuleElectRating for "(...), low"

In [156]:
ob_module_elect_rating = duckdb.sql("""
select
    id,
    'LIC' as ModuleRatingCondition_Value,
    'A' as CurrentAtMaximumPower_Unit,
    "IPmax, low (A)" as CurrentAtMaximumPower_Value,
    'V' as VoltageAtMaximumPower_Unit,
    "VPmax, low (V)" as VoltageAtMaximumPower_Value,
from Dirty;
""")
ob_module_elect_ratings.append(ob_module_elect_rating)
ob_module_elect_rating.show(max_width=120)

┌───────┬──────────────────────┬────────────────────────────┬────────────────────────────┬─────────────────────────────┐
│  id   │ CurrentAtMaximumPo…  │ CurrentAtMaximumPower_Va…  │ VoltageAtMaximumPower_Unit │ VoltageAtMaximumPower_Value │
│ int64 │       varchar        │           double           │          varchar           │           double            │
├───────┼──────────────────────┼────────────────────────────┼────────────────────────────┼─────────────────────────────┤
│     1 │ A                    │                       2.24 │ V                          │                       29.84 │
│     2 │ A                    │                       2.26 │ V                          │                        30.1 │
│     3 │ A                    │                       2.28 │ V                          │                       30.37 │
│     4 │ A                    │                        2.3 │ V                          │                       30.63 │
│     5 │ A                    │

### ModuleElectRating for "(...), NOCT"

In [157]:
ob_module_elect_rating = duckdb.sql("""
select
    id,
    'NOCT' as ModuleRatingCondition_Value,
    'A' as CurrentAtMaximumPower_Unit,
    "IPmax, NOCT (A)" as CurrentAtMaximumPower_Value,
    'V' as VoltageAtMaximumPower_Unit,
    "VPmax, NOCT (V)" as VoltageAtMaximumPower_Value,
from Dirty;
""")
ob_module_elect_ratings.append(ob_module_elect_rating)
ob_module_elect_rating.show(max_width=120)

┌───────┬──────────────────────┬────────────────────────────┬────────────────────────────┬─────────────────────────────┐
│  id   │ CurrentAtMaximumPo…  │ CurrentAtMaximumPower_Va…  │ VoltageAtMaximumPower_Unit │ VoltageAtMaximumPower_Value │
│ int64 │       varchar        │           double           │          varchar           │           double            │
├───────┼──────────────────────┼────────────────────────────┼────────────────────────────┼─────────────────────────────┤
│     1 │ A                    │                       8.81 │ V                          │                       26.34 │
│     2 │ A                    │                       8.88 │ V                          │                       26.57 │
│     3 │ A                    │                       8.96 │ V                          │                       26.81 │
│     4 │ A                    │                       9.04 │ V                          │                       27.04 │
│     5 │ A                    │

### ProdCell.CellTechnologyType

In [18]:
duckdb.sql("""
select distinct Technology from Dirty
""")

┌────────────┐
│ Technology │
│  varchar   │
├────────────┤
│ a-Si/nc    │
│ CdTe       │
│ Thin Film  │
│ Mono-C-si  │
│ NULL       │
│ Mono-c-Si  │
│ Multi-c-Si │
│ CIGS       │
└────────────┘

In [21]:
ob_cell_technology_type = duckdb.sql("""
select
    id,
    case Technology
        when 'a-Si/nc' then 'ASi' -- nc stands for nanocrystalline
        when 'Mono-c-Si' then 'MonoSi'
        when 'Mono-C-si' then 'MonoSi'
        when 'Multi-c-Si' then 'PolySi'
        when 'Thin Film' then 'ThinFilm'
        else Technology
    end as CellTechnologyType_Value,
from Dirty;
""")
duckdb.sql("""
select distinct CellTechnologyType_Value from ob_cell_technology_type
""")

┌──────────────────────────┐
│ CellTechnologyType_Value │
│         varchar          │
├──────────────────────────┤
│ ThinFilm                 │
│ CIGS                     │
│ ASi                      │
│ MonoSi                   │
│ CdTe                     │
│ NULL                     │
│ PolySi                   │
└──────────────────────────┘

### ModuleArea

In [28]:
ob_module_area = duckdb.sql("""
select
    id,
    'sqm' as ModuleArea_Unit,
    "A_c (m2)" as ModuleArea_Value,
from Dirty
""")
ob_module_area

┌───────┬─────────────────┬──────────────────┐
│  id   │ ModuleArea_Unit │ ModuleArea_Value │
│ int64 │     varchar     │      double      │
├───────┼─────────────────┼──────────────────┤
│     1 │ sqm             │            1.627 │
│     2 │ sqm             │            1.627 │
│     3 │ sqm             │            1.627 │
│     4 │ sqm             │            1.627 │
│     5 │ sqm             │            1.627 │
│     6 │ sqm             │            1.638 │
│     7 │ sqm             │            1.638 │
│     8 │ sqm             │            1.638 │
│     9 │ sqm             │            1.638 │
│    10 │ sqm             │            1.638 │
│     · │  ·              │               ·  │
│     · │  ·              │               ·  │
│     · │  ·              │               ·  │
│  9991 │ sqm             │              1.8 │
│  9992 │ sqm             │            1.914 │
│  9993 │ sqm             │            1.926 │
│  9994 │ sqm             │             1.91 │
│  9995 │ sqm

### CellsInSeries

In [31]:
ob_cells_in_series = duckdb.sql("""
select
    id,
    N_s as CellsInSeries_Value,
from Dirty
""")
ob_cells_in_series

┌───────┬─────────────────────┐
│  id   │ CellsInSeries_Value │
│ int64 │        int64        │
├───────┼─────────────────────┤
│     1 │                  60 │
│     2 │                  60 │
│     3 │                  60 │
│     4 │                  60 │
│     5 │                  60 │
│     6 │                  60 │
│     7 │                  60 │
│     8 │                  60 │
│     9 │                  60 │
│    10 │                  60 │
│     · │                   · │
│     · │                   · │
│     · │                   · │
│  9991 │                  60 │
│  9992 │                  72 │
│  9993 │                  72 │
│  9994 │                  72 │
│  9995 │                  72 │
│  9996 │                  60 │
│  9997 │                  60 │
│  9998 │                  60 │
│  9999 │                  72 │
│ 10000 │                  72 │
├───────┴─────────────────────┤
│ ? rows            2 columns │
│ (>9999 rows, 20 shown)      │
└─────────────────────────────┘

### CellStringsParallelQuantity

In [32]:
ob_cells_in_series = duckdb.sql("""
select
    id,
    N_p as CellStringsParallelQuantity_Value,
from Dirty
""")
ob_cells_in_series

┌───────┬───────────────────────────────────┐
│  id   │ CellStringsParallelQuantity_Value │
│ int64 │              double               │
├───────┼───────────────────────────────────┤
│     1 │                               1.0 │
│     2 │                               1.0 │
│     3 │                               1.0 │
│     4 │                               1.0 │
│     5 │                               1.0 │
│     6 │                               1.0 │
│     7 │                               1.0 │
│     8 │                               1.0 │
│     9 │                               1.0 │
│    10 │                               1.0 │
│     · │                                ·  │
│     · │                                ·  │
│     · │                                ·  │
│  9991 │                               2.0 │
│  9992 │                               1.0 │
│  9993 │                               1.0 │
│  9994 │                              NULL │
│  9995 │                         

### IsBIPV

In [13]:
duckdb.sql("""
select distinct BIPV from Dirty
""")

┌─────────┐
│  BIPV   │
│ varchar │
├─────────┤
│ Y       │
│ N       │
│ NULL    │
└─────────┘

In [17]:
ob_is_bipv = duckdb.sql("""
select
    id,
    case BIPV
        when 'Y' then true
        when 'N' then false
        else null
    end as IsBIPV_Value
from Dirty;
""")
duckdb.sql("""
select distinct IsBIPV_Value from ob_is_bipv
""")

┌──────────────┐
│ IsBIPV_Value │
│   boolean    │
├──────────────┤
│ false        │
│ true         │
│ NULL         │
└──────────────┘

### TemperatureNOCT

In [35]:
ob_temperature_noct = duckdb.sql("""
select
    id,
    'Cel' as TemperatureNOCT_Unit,
    "Average NOCT (deg_C)" as TemperatureNOCT_Value,
from Dirty;
""")
ob_temperature_noct

┌───────┬──────────────────────┬───────────────────────┐
│  id   │ TemperatureNOCT_Unit │ TemperatureNOCT_Value │
│ int64 │       varchar        │        double         │
├───────┼──────────────────────┼───────────────────────┤
│     1 │ Cel                  │                  47.4 │
│     2 │ Cel                  │                  47.4 │
│     3 │ Cel                  │                  47.4 │
│     4 │ Cel                  │                  47.4 │
│     5 │ Cel                  │                  47.4 │
│     6 │ Cel                  │                  47.6 │
│     7 │ Cel                  │                  47.6 │
│     8 │ Cel                  │                  47.6 │
│     9 │ Cel                  │                  46.0 │
│    10 │ Cel                  │                  47.6 │
│     · │  ·                   │                    ·  │
│     · │  ·                   │                    ·  │
│     · │  ·                   │                    ·  │
│  9991 │ Cel                  

### TemperatureCoefficientMaximumPower

In [38]:
ob_temperature_coefficient_maximum_power = duckdb.sql("""
select
    id,
    'percent_per_Cel' as TemperatureCoefficientMaximumPower_Unit,
    "γPmax (percent/deg_C)" as TemperatureCoefficientMaximumPower_Value,
from Dirty;
""")
ob_temperature_coefficient_maximum_power

┌───────┬─────────────────────────────────────────┬──────────────────────────────────────────┐
│  id   │ TemperatureCoefficientMaximumPower_Unit │ TemperatureCoefficientMaximumPower_Value │
│ int64 │                 varchar                 │                  double                  │
├───────┼─────────────────────────────────────────┼──────────────────────────────────────────┤
│     1 │ percent_per_Cel                         │                                  -0.4509 │
│     2 │ percent_per_Cel                         │                                  -0.4509 │
│     3 │ percent_per_Cel                         │                                  -0.4509 │
│     4 │ percent_per_Cel                         │                                  -0.4509 │
│     5 │ percent_per_Cel                         │                                  -0.4509 │
│     6 │ percent_per_Cel                         │                                  -0.4501 │
│     7 │ percent_per_Cel                         

### TemperatureCoefficientShortCircuitCurrent

In [47]:
duckdb.sql("""
select distinct
    "αIsc  (percent/deg_C)" from Dirty
where try_cast("αIsc  (percent/deg_C)" as double) is null;
""")

┌───────────────────────┐
│ αIsc  (percent/deg_C) │
│        varchar        │
├───────────────────────┤
│ NULL                  │
└───────────────────────┘

In [53]:
ob_temperature_coefficient_short_circuit_current = duckdb.sql("""
select
    id,
    'percent_per_Cel' as TemperatureCoefficientShortCircuitCurrent_Unit,
    cast("αIsc  (percent/deg_C)" as double) as TemperatureCoefficientShortCircuitCurrent_Value,
from Dirty;
""")
ob_temperature_coefficient_short_circuit_current

┌───────┬────────────────────────────────────────────────┬─────────────────────────────────────────────────┐
│  id   │ TemperatureCoefficientShortCircuitCurrent_Unit │ TemperatureCoefficientShortCircuitCurrent_Value │
│ int64 │                    varchar                     │                     double                      │
├───────┼────────────────────────────────────────────────┼─────────────────────────────────────────────────┤
│     1 │ percent_per_Cel                                │                                          0.0521 │
│     2 │ percent_per_Cel                                │                                          0.0521 │
│     3 │ percent_per_Cel                                │                                          0.0521 │
│     4 │ percent_per_Cel                                │                                          0.0521 │
│     5 │ percent_per_Cel                                │                                          0.0521 │
│     6 │ percent_p

### TemperatureCoefficientOpenCircuitVoltage

In [54]:
duckdb.sql("""
select distinct
    "βVoc (percent/deg_C)" from Dirty
where try_cast("βVoc (percent/deg_C)" as double) is null;
""")

┌──────────────────────┐
│ βVoc (percent/deg_C) │
│       varchar        │
├──────────────────────┤
│  -0.2794             │
│ NULL                 │
└──────────────────────┘

In [55]:
ob_temperature_coefficient_open_circuit_voltage = duckdb.sql("""
select
    id,
    'percent_per_Cel' as TemperatureCoefficientOpenCircuitVoltage_Unit,
    cast(trim("βVoc (percent/deg_C)") as double) as TemperatureCoefficientOpenCircuitVoltage_Value,
from Dirty;
""")
ob_temperature_coefficient_open_circuit_voltage

┌───────┬───────────────────────────────────────────────┬────────────────────────────────────────────────┐
│  id   │ TemperatureCoefficientOpenCircuitVoltage_Unit │ TemperatureCoefficientOpenCircuitVoltage_Value │
│ int64 │                    varchar                    │                     double                     │
├───────┼───────────────────────────────────────────────┼────────────────────────────────────────────────┤
│     1 │ percent_per_Cel                               │                                        -0.3137 │
│     2 │ percent_per_Cel                               │                                        -0.3137 │
│     3 │ percent_per_Cel                               │                                        -0.3137 │
│     4 │ percent_per_Cel                               │                                        -0.3137 │
│     5 │ percent_per_Cel                               │                                        -0.3137 │
│     6 │ percent_per_Cel            

### TemperatureCoefficientMaxPowerCurrent

In [60]:
duckdb.sql("""
select distinct
    "αIpmax (percent/deg_C)" from Dirty
where try_cast("αIpmax (percent/deg_C)" as double) is null;
""")

┌────────────────────────┐
│ αIpmax (percent/deg_C) │
│        varchar         │
├────────────────────────┤
│                        │
│ NULL                   │
└────────────────────────┘

In [63]:
ob_temperature_coefficient_max_power_current = duckdb.sql("""
select
    id,
    'percent_per_Cel' as TemperatureCoefficientMaxPowerCurrent_Unit,
    try_cast("αIpmax (percent/deg_C)" as double) as TemperatureCoefficientMaxPowerCurrent_Value,
from Dirty;
""")
ob_temperature_coefficient_max_power_current

┌───────┬────────────────────────────────────────────┬─────────────────────────────────────────────┐
│  id   │ TemperatureCoefficientMaxPowerCurrent_Unit │ TemperatureCoefficientMaxPowerCurrent_Value │
│ int64 │                  varchar                   │                   double                    │
├───────┼────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     1 │ percent_per_Cel                            │                                      0.0113 │
│     2 │ percent_per_Cel                            │                                      0.0113 │
│     3 │ percent_per_Cel                            │                                      0.0113 │
│     4 │ percent_per_Cel                            │                                      0.0113 │
│     5 │ percent_per_Cel                            │                                      0.0113 │
│     6 │ percent_per_Cel                            │                                   -0

### TemperatureCoefficientMaxPowerVoltage

In [64]:
duckdb.sql("""
select distinct
    "βVpmax (percent/deg_C)" from Dirty
where try_cast("βVpmax (percent/deg_C)" as double) is null;
""")

┌────────────────────────┐
│ βVpmax (percent/deg_C) │
│        varchar         │
├────────────────────────┤
│                        │
│ NULL                   │
└────────────────────────┘

In [65]:
ob_temperature_coefficient_max_power_voltage = duckdb.sql("""
select
    id,
    'percent_per_Cel' as TemperatureCoefficientMaxPowerVoltage_Unit,
    try_cast("βVpmax (percent/deg_C)" as double) as TemperatureCoefficientMaxPowerVoltage_Value,
from Dirty;
""")
ob_temperature_coefficient_max_power_voltage

┌───────┬────────────────────────────────────────────┬─────────────────────────────────────────────┐
│  id   │ TemperatureCoefficientMaxPowerVoltage_Unit │ TemperatureCoefficientMaxPowerVoltage_Value │
│ int64 │                  varchar                   │                   double                    │
├───────┼────────────────────────────────────────────┼─────────────────────────────────────────────┤
│     1 │ percent_per_Cel                            │                                     -0.4632 │
│     2 │ percent_per_Cel                            │                                     -0.4632 │
│     3 │ percent_per_Cel                            │                                     -0.4632 │
│     4 │ percent_per_Cel                            │                                     -0.4632 │
│     5 │ percent_per_Cel                            │                                     -0.4632 │
│     6 │ percent_per_Cel                            │                                     

### Dimension.(Width|Height)

In [66]:
ob_width_height = duckdb.sql("""
select
    id,
    'm' as Width_Unit,
    "Short Side (m)" as Width_Value,
    'm' as Height_Unit,
    "Long Side (m)" as Height_Value,
from Dirty;
""")
ob_width_height

┌───────┬────────────┬─────────────┬─────────────┬──────────────┐
│  id   │ Width_Unit │ Width_Value │ Height_Unit │ Height_Value │
│ int64 │  varchar   │   double    │   varchar   │    double    │
├───────┼────────────┼─────────────┼─────────────┼──────────────┤
│     1 │ m          │       0.992 │ m           │         1.64 │
│     2 │ m          │       0.992 │ m           │         1.64 │
│     3 │ m          │       0.992 │ m           │         1.64 │
│     4 │ m          │       0.992 │ m           │         1.64 │
│     5 │ m          │       0.992 │ m           │         1.64 │
│     6 │ m          │       0.995 │ m           │        1.646 │
│     7 │ m          │       0.995 │ m           │        1.646 │
│     8 │ m          │       0.995 │ m           │        1.646 │
│     9 │ m          │       0.995 │ m           │        1.646 │
│    10 │ m          │       0.995 │ m           │        1.646 │
│     · │ ·          │          ·  │ ·           │           ·  │
│     · │ 

### EfficiencyBatteryRoundTrip

In [198]:
duckdb.sql("""
select distinct
    'decimal_percent' as EfficiencyBatteryRoundTrip_Unit,
    "Manufacturer Declared Roundtrip Efficiency (%, Ac-AC)1" as EfficiencyBatteryRoundTrip_Value,
from Dirty;
""").show(max_width=120)

┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│ EfficiencyBatteryR…  │                               EfficiencyBatteryRoundTrip_Value                                │
│       varchar        │                                            varchar                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ decimal_percent      │ 89 [JKS-R-7P12-US]\n89 [JKS-R-11P12-US]                                                       │
│ decimal_percent      │ Inverter 18kW]                                                                                │
│ decimal_percent      │ 98 [Sol-Ark 5K]\n98 [Sol-Ark 8K]\n98 [Sol-Ark 12K]\n98 [Sol-Ark 15K]                          │
│ decimal_percent      │ 89 [Sanctuary 3 \nInverter 12kW]\n89 [Sanctuary 3 \nInverter 18kW]\n89 [Sanctuary 3 \nInver…  │
│ decimal_percent      │ 81.04 [

In [196]:
duckdb.sql("""
select distinct
    "Certified JA12 Control  Strategies1"
from Dirty;
""").show(max_width=120)

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                         Certified JA12 Control  Strategies1                                          │
│                                                       varchar                                                        │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Sol-Ark 5K - Basic, TOU, Advanced\nSol-Ark 8K - Basic, TOU, Advanced\nSol-Ark 12K - Basic, TOU, Advanced\nSol-Ark …  │
│ No Information Submitted                                                                                             │
│ Inverter 24kW - Basic, TOU, Advance DR                                                                               │
│ Multiplus & Multiplus II & MP-II GX 48/3000/35 - Basic, TOU, Advance DR\nMultiplus & Multiplus II & MP-II GX 48/50…  │
│ Basic, TOU, Advanced DR       

In [200]:
duckdb.sql("""
select distinct "Declaration for JA12 Submitted1" from Dirty;
""").show(max_width=120)

┌─────────────────────────────────┐
│ Declaration for JA12 Submitted1 │
│             varchar             │
├─────────────────────────────────┤
│ Y (2022)                        │
│ N                               │
└─────────────────────────────────┘



In [201]:
duckdb.sql("""
select distinct Notes from Dirty;
""").show(max_width=120)

┌───────────────────────────────────────────────┐
│                     Notes                     │
│                    varchar                    │
├───────────────────────────────────────────────┤
│ Formerly Stored Energy Inc, PF5-LFP14400-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP33600-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP09600-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP28800-2A01 │
│ NULL                                          │
│ Formerly Stored Energy Inc, PF5-LFP19200-2A01 │
│ Formerly Stored Energy Inc, PF5-LFP38400-2A01 │
│                                               │
│ Formerly Stored Energy Inc, PF5-LFP24000-2A01 │
└───────────────────────────────────────────────┘



In [202]:
duckdb.sql("""
select distinct "CEC Listing Date" from Dirty;
""").show(max_width=120)

┌──────────────────┐
│ CEC Listing Date │
│       date       │
├──────────────────┤
│ 2024-11-12       │
│ 2019-09-27       │
│ 2021-10-21       │
│ 2024-08-12       │
│ 2024-10-01       │
│ 2023-10-11       │
│ 2025-09-02       │
│ 2023-12-01       │
│ 2023-11-01       │
│ 2025-12-11       │
│     ·            │
│     ·            │
│     ·            │
│ 2023-02-13       │
│ 2025-11-12       │
│ 2022-04-21       │
│ 2022-06-21       │
│ 2026-01-21       │
│ 2025-12-22       │
│ 2025-08-11       │
│ 2023-05-01       │
│ 2026-02-11       │
│ 2023-02-21       │
├──────────────────┤
│     121 rows     │
│    (20 shown)    │
└──────────────────┘



In [203]:
duckdb.sql("""
select distinct "Last Update" from Dirty;
""").show(max_width=120)

┌─────────────┐
│ Last Update │
│    date     │
├─────────────┤
│ 2025-12-11  │
│ 2023-12-01  │
│ 2023-07-03  │
│ 2025-08-21  │
│ 2024-04-02  │
│ 2023-05-11  │
│ 2024-09-11  │
│ 2026-02-02  │
│ NULL        │
│ 2024-06-03  │
│ 2024-11-01  │
│ 2024-01-02  │
│ 2024-03-01  │
│ 2023-05-22  │
│ 2019-12-23  │
│ 2023-09-21  │
├─────────────┤
│   16 rows   │
└─────────────┘



# Ignored columns

In [73]:
duckdb.sql("""
select distinct Notes from Dirty;
""")

┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│                                            Notes                                             │
│                                           varchar                                            │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Formerly GEP NEW ENERGY VIETNAM COMPANY LIMITED GEP-JfMc385MH                                │
│ Formerly GEP NEW ENERGY VIETNAM COMPANY LIMITED GEP-BfMc435MHT                               │
│ Formerly GEP NEW ENERGY VIETNAM COMPANY LIMITED GEP-AfMc540MHT                               │
│ Formerly GEP NEW ENERGY VIETNAM COMPANY LIMITED GEP-AfMc550MHT                               │
│ Formerly listed under Hanwha Q CELLS                                                         │
│ Also sold as Renesola Jiangsu JC310M-24/Azs                                                  │
│ This component is specific t

In [67]:
duckdb.sql("""
select distinct Mounting from Dirty;
""")

┌──────────┐
│ Mounting │
│ varchar  │
├──────────┤
│ BIPV     │
│ Rack     │
│ NULL     │
│          │
└──────────┘

In [68]:
duckdb.sql("""
select distinct Type from Dirty;
""")

┌────────────┐
│    Type    │
│  varchar   │
├────────────┤
│ NULL       │
│ Flat Plate │
└────────────┘

In [69]:
duckdb.sql("""
select distinct "Geometric Multiplier" from Dirty;
""")

┌──────────────────────┐
│ Geometric Multiplier │
│        double        │
├──────────────────────┤
│                  1.0 │
│                 NULL │
└──────────────────────┘

In [70]:
duckdb.sql("""
select distinct "P2/Pref" from Dirty;
""")

┌─────────────┐
│   P2/Pref   │
│   double    │
├─────────────┤
│ 0.247189416 │
│ 0.197982273 │
│ 0.204770847 │
│ 0.203849508 │
│ 0.189502179 │
│ 0.189820364 │
│  0.19658209 │
│ 0.188322713 │
│ 0.182702717 │
│ 0.188625372 │
│      ·      │
│      ·      │
│      ·      │
│ 0.197955624 │
│ 0.195099004 │
│ 0.194900053 │
│ 0.196779742 │
│ 0.191674952 │
│ 0.192431979 │
│ 0.192448769 │
│ 0.192552001 │
│ 0.192437083 │
│ 0.192360233 │
├─────────────┤
│  3454 rows  │
│ (20 shown)  │
└─────────────┘

In [71]:
duckdb.sql("""
select distinct "CEC Listing Date" from Dirty;
""")

┌──────────────────┐
│ CEC Listing Date │
│       date       │
├──────────────────┤
│ 2024-08-01       │
│ 2024-07-22       │
│ 2022-04-11       │
│ 2024-06-21       │
│ 2023-10-23       │
│ 2025-05-12       │
│ 2017-06-01       │
│ 2018-07-02       │
│ 2020-05-21       │
│ 2023-05-11       │
│     ·            │
│     ·            │
│     ·            │
│ 2019-08-01       │
│ 2021-11-01       │
│ 2024-09-23       │
│ 2023-02-13       │
│ 2017-10-16       │
│ 2025-11-12       │
│ 2022-03-01       │
│ 2023-12-21       │
│ 2020-09-01       │
│ 2020-06-11       │
├──────────────────┤
│     278 rows     │
│    (20 shown)    │
└──────────────────┘

In [72]:
duckdb.sql("""
select distinct "Last Update" from Dirty;
""")

┌─────────────┐
│ Last Update │
│    date     │
├─────────────┤
│ 2018-07-02  │
│ 2025-07-01  │
│ 2024-07-22  │
│ 2024-06-21  │
│ 2018-09-17  │
│ 2019-03-01  │
│ 2020-05-21  │
│ 2022-05-23  │
│ 2023-05-11  │
│ 2021-01-11  │
│     ·       │
│     ·       │
│     ·       │
│ 2025-11-12  │
│ 2019-05-15  │
│ 2020-06-11  │
│ 2026-02-11  │
│ 2020-11-02  │
│ 2019-12-02  │
│ 2021-09-13  │
│ 2025-08-11  │
│ 2025-12-22  │
│ 2017-11-15  │
├─────────────┤
│  158 rows   │
│ (20 shown)  │
└─────────────┘

# Building clean data